この問題は、リソースの消費を平均80%以内にするための物理サーバの必要台数を計算するものです。問題の解説にあるように、物理サーバが8台停止してもリソース消費が80%以内に収まる条件を満たすための台数を求めるための公式が使われています。

問題の条件から次のように式を導きます。

### 解説

1. **平常時のリソース消費:**
   - 平常時の物理サーバの台数を \( N \) 台とする。
   - 平常時のリソース消費は60%なので、消費リソースは \( 0.6N \)。

2. **物理サーバが8台停止した場合:**
   - 停止後の物理サーバの台数は \( N-8 \) 台。
   - 80%以内に抑えるためのリソース消費は \( 0.8(N-8) \)。

3. **リソースの再配分:**
   - 停止後の稼働中のサーバにリソースが均等に再配分されるため、次の等式が成立します。
   \[
   0.6N = 0.8(N - 8)
   \]

4. **等式の解:**
   - 上記の等式を解くことで必要な物理サーバの台数 \( N \) を求めます。
   \[
   0.6N = 0.8N - 6.4
   \]
   \[
   0.6N - 0.8N = -6.4
   \]
   \[
   -0.2N = -6.4
   \]
   \[
   N = 32
   \]

### 結論
物理サーバが1台も停止していないときの最低台数は32台です。この解を基に、選択肢の中で正しい答えは「ア 32」となります。

以上の計算を元にして、必要な物理サーバの台数を求めることができます。

In [6]:
import numpy as np

for _ in range(10):
    print(np.random.rand() * 10)

6.241085016069416
4.498714782464683
8.597776156696817
5.207939301696566
5.542405480248166
8.562762435313148
0.3256940526641894
5.6674627918473774e-05
3.785764973048713
6.91797919906326


In [7]:
from sqlalchemy import create_engine, Column, Integer, String, Float, MetaData, Table
from sqlalchemy.orm import sessionmaker

# メモリ内のSQLiteデータベースを作成
engine = create_engine('sqlite:///example.db')
metadata = MetaData()

# テーブルの定義
olap_table = Table('olap', metadata,
    Column('id', Integer, primary_key=True),
    Column('year', Integer),
    Column('region', String),
    Column('category', String),
    Column('sales', Float)
)

# テーブルの作成
metadata.create_all(engine)

# セッションの作成
Session = sessionmaker(bind=engine)
session = Session()

# サンプルデータの挿入
sample_data = [
    {'year': 2021, 'region': 'North', 'category': 'Electronics', 'sales': 1000.0},
    {'year': 2021, 'region': 'South', 'category': 'Electronics', 'sales': 1500.0},
    {'year': 2021, 'region': 'North', 'category': 'Furniture', 'sales': 700.0},
    {'year': 2021, 'region': 'South', 'category': 'Furniture', 'sales': 800.0},
    {'year': 2022, 'region': 'North', 'category': 'Electronics', 'sales': 1100.0},
    {'year': 2022, 'region': 'South', 'category': 'Electronics', 'sales': 1600.0},
    {'year': 2022, 'region': 'North', 'category': 'Furniture', 'sales': 750.0},
    {'year': 2022, 'region': 'South', 'category': 'Furniture', 'sales': 850.0},
]

# データを挿入
session.execute(olap_table.insert(), sample_data)
session.commit()

- pandasでの処理

In [8]:
import pandas as pd

# データベースからデータを取得
df = pd.read_sql_table('olap', con=engine)

# データの表示
print("Data using pandas:")
print(df)

# ロールアップ: 年ごとの売上を集計
rollup_df = df.groupby('year')['sales'].sum().reset_index()
print("\nRoll-up using pandas:")
print(rollup_df)

Data using pandas:
   id  year region     category   sales
0   1  2021  North  Electronics  1000.0
1   2  2021  South  Electronics  1500.0
2   3  2021  North    Furniture   700.0
3   4  2021  South    Furniture   800.0
4   5  2022  North  Electronics  1100.0
5   6  2022  South  Electronics  1600.0
6   7  2022  North    Furniture   750.0
7   8  2022  South    Furniture   850.0

Roll-up using pandas:
   year   sales
0  2021  4000.0
1  2022  4300.0


In [10]:
!pip install polars
!pip install --upgrade pip


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached pip-24.1.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.1.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


- Polarsの場合

In [13]:
import polars as pl

# Polarsを使ってデータベースからデータを読み込む
query = "SELECT * FROM olap"
df_polars = pl.read_database(query=query, connection=engine.connect())

# データの表示
print("Data using polars:")
df_polars

Data using polars:


id,year,region,category,sales
i64,i64,str,str,f64
1,2021,"""North""","""Electronics""",1000.0
2,2021,"""South""","""Electronics""",1500.0
3,2021,"""North""","""Furniture""",700.0
4,2021,"""South""","""Furniture""",800.0
5,2022,"""North""","""Electronics""",1100.0
6,2022,"""South""","""Electronics""",1600.0
7,2022,"""North""","""Furniture""",750.0
8,2022,"""South""","""Furniture""",850.0


In [19]:
import polars as pl

# サンプルデータの作成
data = {
    "year": [2021, 2021, 2022, 2022],
    "region": ["North", "South", "North", "South"],
    "sales": [1000.0, 1500.0, 1100.0, 1600.0]
}

# DataFrameの作成
df = pl.DataFrame(data)

# DataFrameをLazyFrameに変換
lazy_df = df.lazy()

# グループ化と集約操作を遅延評価で定義
rollup_df = (
    lazy_df
    .group_by("year")
    .agg(pl.col("sales").sum().alias("total_sales"))
)

# 結果を収集（実行）
result = rollup_df.collect()

result

year,total_sales
i64,f64
2021,2500.0
2022,2700.0


In [20]:
# Polarsを使ってデータベースからデータを読み込む
query = "SELECT * FROM olap"
df_polars = pl.read_database(query=query, connection=engine.connect())

# データの表示
print("Data using polars:")
print(df_polars)

# DataFrameをLazyFrameに変換
lazy_df = df_polars.lazy()

# グループ化と集約操作を遅延評価で定義
rollup_df = (
    lazy_df
    .group_by("year")
    .agg(pl.col("sales").sum().alias("total_sales"))
)

# 結果を収集（実行）
result = rollup_df.collect()

print("\nRoll-up using polars:")
print(result)

Data using polars:
shape: (8, 5)
┌─────┬──────┬────────┬─────────────┬────────┐
│ id  ┆ year ┆ region ┆ category    ┆ sales  │
│ --- ┆ ---  ┆ ---    ┆ ---         ┆ ---    │
│ i64 ┆ i64  ┆ str    ┆ str         ┆ f64    │
╞═════╪══════╪════════╪═════════════╪════════╡
│ 1   ┆ 2021 ┆ North  ┆ Electronics ┆ 1000.0 │
│ 2   ┆ 2021 ┆ South  ┆ Electronics ┆ 1500.0 │
│ 3   ┆ 2021 ┆ North  ┆ Furniture   ┆ 700.0  │
│ 4   ┆ 2021 ┆ South  ┆ Furniture   ┆ 800.0  │
│ 5   ┆ 2022 ┆ North  ┆ Electronics ┆ 1100.0 │
│ 6   ┆ 2022 ┆ South  ┆ Electronics ┆ 1600.0 │
│ 7   ┆ 2022 ┆ North  ┆ Furniture   ┆ 750.0  │
│ 8   ┆ 2022 ┆ South  ┆ Furniture   ┆ 850.0  │
└─────┴──────┴────────┴─────────────┴────────┘

Roll-up using polars:
shape: (2, 2)
┌──────┬─────────────┐
│ year ┆ total_sales │
│ ---  ┆ ---         │
│ i64  ┆ f64         │
╞══════╪═════════════╡
│ 2021 ┆ 4000.0      │
│ 2022 ┆ 4300.0      │
└──────┴─────────────┘


In [7]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, select
from sqlalchemy.orm import sessionmaker
import pandas as pd

# メモリ上のSQLiteデータベースを作成
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()

# サンプルのテーブルを作成
sample_table = Table(
    'sample_table', metadata,
    Column('id', Integer, primary_key=True),
    Column('data', String)
)

metadata.create_all(engine)

# 接続を作成
connection = engine.connect()
Session = sessionmaker(bind=connection)
session = Session()

# データの挿入
connection.execute(sample_table.insert(), [
    {'id': 1, 'data': 'データ1'},
    {'id': 2, 'data': 'データ2'},
    {'id': 3, 'data': 'データ3'}
])

# トランザクションの開始
session.begin()

try:
    # データの更新
    session.execute(
        sample_table.update().where(sample_table.c.id == 1).values(data='更新されたデータ1')
    )
    session.commit()  # トランザクションのコミット

    # トランザクションがコミットされた後にデータを読み込む
    result = connection.execute(select(sample_table))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print(df)
except:
    print('Rollback')
    session.rollback()  # ロールバック
finally:
    session.close()
    connection.close()

   id       data
0   1  更新されたデータ1
1   2       データ2
2   3       データ3
